In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = 'cointegrated/rubert-tiny-toxicity'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
    model.cuda()
    
def text2toxicity(text, aggregate=True):
    """ Calculate toxicity of a text (if aggregate=True) or a vector of toxicity aspects (if aggregate=False)"""
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()
    if isinstance(text, str):
        proba = proba[0]
    if aggregate:
        return 1 - proba.T[0] * (1 - proba.T[-1])
    return proba

Downloading:   0%|          | 0.00/377 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/235k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/457k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/957 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/45.0M [00:00<?, ?B/s]

In [2]:
import pandas as pd

csv_data = pd.read_csv('punc_train.tsv', sep='\t', header=0)
csv_data = csv_data[['punc_phrase', 'punc_context', 'label']]
csv_data.columns = ['phrase', 'context', 'label']
csv_data.drop_duplicates(['context', 'phrase', 'label'], inplace=True)

In [37]:
csv_data['context'] = csv_data['context'].apply(lambda x: eval(x))

In [29]:
csv_data['toxicity_phrase'] = csv_data['phrase'].apply(lambda x: text2toxicity(x, True))

In [41]:
def toxicity_of_context(context):
    toxicities = []
    for x in context:
        user_repl = x['user']
        if user_repl:
            repl_toxicity = text2toxicity(user_repl, True)
            toxicities.append(repl_toxicity)
        else:
            continue
    mean_toxicity = np.mean(toxicities) if toxicities else 0
    return mean_toxicity

In [42]:
import numpy as np


csv_data['toxicity_user_context'] = csv_data['context'].apply(lambda x: toxicity_of_context(x))

In [52]:
csv_data.sort_values(by='toxicity_user_context', ascending=False)['context'].values[:10]

array([list([{'user': '', 'marusia': ''}, {'user': '', 'marusia': ' [Добрый вечер, Амина!] | [date: Воскресенье, 14 августа] | [temp: 32] | [ Новости кино и компьютерных игр (⚡DTF) {Включи новости DTF}, Погода {Погода в понедельник}, Давай играть в загадки {Давай играть в загадки}, Почитаю свежие посты (Что нового ВКонтакте?) {Почитай ленту VK}, Игра Города {Давай сыграем в города?}, Радио {Включи Эхо Москвы}, Включу плейлист для тренировки (Время спорта! 💪) {Включи плейлист для спорта}, Новые источники в «Новостях» (Включу Лента.ру) {Включи новости Лента.ру}, Таймер {Поставь таймер на полчаса}]'}, {'user': '', 'marusia': ' [Добрый вечер, Амина!] | [date: Воскресенье, 14 августа] | [temp: 32] | [ Новости кино и компьютерных игр (⚡DTF) {Включи новости DTF}, Погода {Брать ли зонт завтра?}, Загадки {Давай сыграем в загадки}, Давай сыграем в города {Давай сыграем в города}, Радио {Включи Весна ФМ}, Новые источники в «Новостях» (Включу Лента.ру) {Включи новости Лента.ру}, Таймер {Таймер на 

In [47]:
csv_data[csv_data['toxicity_user_context'] != 0].corr()

,label,toxicity_phrase,toxicity_user_context
label,1.00000,-0.024380,-0.021680
toxicity_phrase,-0.02438,1.000000,0.223775
toxicity_user_context,-0.02168,0.223775,1.000000


In [44]:
csv_data.corr()

,label,toxicity_phrase,toxicity_user_context
label,1.000000,-0.024029,-0.023225
toxicity_phrase,-0.024029,1.000000,0.223819
toxicity_user_context,-0.023225,0.223819,1.000000


In [36]:
csv_data[['phrase', 'label', 'toxicity_phrase']].sort_values(by='toxicity_phrase', ascending=False).head(50)

,phrase,label,toxicity_phrase
9391,"Я убью тебя, тварь.",0,0.999921
26144,"Если она просто тебе по со сукам, лицо тебя ра...",1,0.999908
8884,Писюнчик. писюнчик. Ты любишь писюнчик.,0,0.999867
28728,Почему она так долго с ними? Почему Денис не т...,1,0.999837
21393,"Маруся, ты проститутка.",0,0.999823
25667,"Ты писюнчик, бобряночка.",0,0.999807
21615,"Тут говорят, что ты овца тупорылая.",0,0.999805
9913,"Ты баран, дебил кусок.",0,0.999794
14837,Да ты вообще конченая дебилка. Конченая. конче...,0,0.999793
18466,Вот иди ты в задницу.,0,0.999784


In [ ]:
csv_data['']

In [27]:
text2toxicity('Ты милаш', True)

0.024522583384078755

In [21]:
text2toxicity(csv_data['phrase'][150], False)[0]

0.99975103

In [14]:
csv_data['phrase'][150]

'Скажи это голосом учителя, пожалуйста.'